# Рак Алексей

In [1]:
from Bio import SeqIO
import pandas as pd

In [6]:
data_path = '/home/alex/Yandex.Disk/BSU/Bio2/Практика 2/data/'

# Задание 1

In [14]:
genes = {}

def get_id(s):
    idname = s.split('#')[0]
    return idname[1:-1]

with open(data_path + 'SK926/SK926.fna') as f:
    line = f.readline()
    idname = get_id(line)
    gen = ''
    while line:
        if line[0] == '>':
            genes[idname] = gen
            gen = ''
            idname = get_id(line)
        else:
            gen += line[:-1]
        line = f.readline()
    print('genes size: ',len(genes))

genes size:  437618


In [15]:
dic = {}
with open(data_path + 'SK926/SK926.blast_swissprot') as f:
    for line in f:
        t = line.split('\t')
        dic[t[1]] = t[0]
print('dic size',len(dic))

dic size 45658


In [16]:
aname = 'Alphaproteobacteria'
cname = 'Clostridiales'
A = set()
C = set()
with open(data_path + 'SK926/swissprot_db.dat') as f:
    for line in f:
        t = line.split('\t')
        label = t[0];
        if t[2].find(aname) != -1:
            A.add(label)
        if t[2].find(cname) != -1:
            C.add(label)
print(aname,' founded: ',len(A))
print(cname,' founded: ',len(C))

Alphaproteobacteria  founded:  1757
Clostridiales  founded:  4320


In [18]:
def get_codons(s, codons, count):
    for i in range(0, len(s) - 2, 3):
        if s[i:i + 3] in codons:
            codons[s[i:i+3]] += 1
        else:
            codons[s[i:i+3]] = 0
        count += 1
    return count

In [19]:
A_codons = {}
C_codons = {}
A_count = 0
C_count = 0
for a in A:
    if a in dic and dic[a] in genes:
        A_count = get_codons(genes[dic[a]], A_codons, A_count)
for c in C:
    if c in dic and dic[c] in genes:
        C_count = get_codons(genes[dic[c]], C_codons, C_count)
        
print('A codons: ', A_count)
print('C codons: ', C_count)

A codons:  280199
C codons:  563167


In [25]:
print ("\tA\tC\t")
for key in A_codons.keys():
    print("%s\t%.4f\t%.4f" % (key,A_codons[key]/A_count,C_codons[key]/C_count))

	A	C	
ACT	0.0083	0.0083
AGA	0.0073	0.0127
ATG	0.0289	0.0298
GAT	0.0287	0.0327
CGG	0.0088	0.0063
CCT	0.0078	0.0075
GTG	0.0241	0.0214
GAA	0.0377	0.0414
GGT	0.0165	0.0193
CTG	0.0378	0.0321
AAT	0.0198	0.0197
GCC	0.0300	0.0231
TAC	0.0137	0.0136
CAA	0.0082	0.0054
TCC	0.0142	0.0120
CCC	0.0103	0.0091
AGC	0.0128	0.0129
TGA	0.0008	0.0007
AAA	0.0291	0.0320
TGG	0.0097	0.0062
ATA	0.0103	0.0115
GTT	0.0154	0.0209
GAG	0.0296	0.0346
GAC	0.0270	0.0283
CAT	0.0112	0.0104
AGG	0.0035	0.0033
CTT	0.0176	0.0209
TCG	0.0085	0.0052
CGT	0.0114	0.0116
ACC	0.0191	0.0184
AAG	0.0282	0.0386
TAG	0.0003	0.0004
CAG	0.0251	0.0251
GTC	0.0168	0.0152
GCA	0.0217	0.0268
CCA	0.0047	0.0052
GGA	0.0191	0.0189
CCG	0.0178	0.0140
ATC	0.0317	0.0324
ATT	0.0238	0.0248
GCG	0.0204	0.0171
TTG	0.0130	0.0080
TAA	0.0014	0.0018
ACA	0.0126	0.0160
TTC	0.0210	0.0163
TTT	0.0193	0.0176
TCT	0.0083	0.0079
GGG	0.0088	0.0063
TAT	0.0203	0.0198
TGC	0.0092	0.0096
TGT	0.0054	0.0056
CAC	0.0092	0.0090
GCT	0.0167	0.0183
CTA	0.0024	0.0015
TCA	0.0063	0.0071
GTA	

# Задание 2

In [26]:
ptt_file1 = data_path + 'NC_012659.ptt'
ptt_file2 = data_path + 'NC_014500.ptt'

In [27]:
class Gene:
    def __init__(self, line):
        self.data = line
        self.name = self.data[4]
        
        
    def find(self, line):
        return self.data[-1].find(line) != -1
        
        
    def __str__(self):
        return self.name + ' - ' + self.data[-1]

In [28]:
class Genome:
    def __init__(self, path):
        with open(path) as f:
            lines = f.readlines()
        self.name = lines[0].split(',')[0]
        self.data = []
        for line in lines[3:]:
            gene = Gene(line[:-1].split('\t'))
            if gene.name != '-':
                self.data.append(gene)
            
    def __len__(self):
        return len(self.data)

    
    def __str__(self):
        return self.name

        
    def get_functions(self, s):
        for gene in self.data:
            if gene.find(s):
                yield gene
                
    
    def get_common_gene_names(self, other):
        names = set()
        for gene in self.data:
            names.add(gene.name)
        ans = set()
        for gene in other.data:
            if gene.name in names:
                ans.add(gene.name)
        return ans

In [29]:
genome1 = Genome(ptt_file1)
genome2 = Genome(ptt_file2)
print(genome1, 'has', len(genome1), 'genes')
print(genome2, 'has', len(genome2), 'genes')
print()
search_string = 'chaperone'
for gene in genome1.get_functions(search_string):
    print(genome1, 'has', gene)
print()
for gene in genome2.get_functions(search_string):
    print(genome2, 'has', gene)
print()
common_genes = genome1.get_common_gene_names(genome2)
print('%s ans %s have %d genes in common' % (genome1, genome2, len(common_genes)))

Bacillus anthracis str. A0248 has 1158 genes
Dickeya dadantii 3937 chromosome has 2546 genes

Bacillus anthracis str. A0248 has csaA - chaperone CsaA
Bacillus anthracis str. A0248 has narJ - nitrate reductase molybdenum cofactor assembly chaperone
Bacillus anthracis str. A0248 has hscC - chaperone protein hscC
Bacillus anthracis str. A0248 has dnaJ - chaperone protein DnaJ
Bacillus anthracis str. A0248 has dnaK - molecular chaperone DnaK

Dickeya dadantii 3937 chromosome has ibpA - heat shock chaperone
Dickeya dadantii 3937 chromosome has ibpB - heat shock chaperone
Dickeya dadantii 3937 chromosome has hslU - molecular chaperone and ATPase of HslUV protease
Dickeya dadantii 3937 chromosome has hlpA - periplasmic chaperone
Dickeya dadantii 3937 chromosome has htpG - molecular chaperone HSP90 family
Dickeya dadantii 3937 chromosome has clpA - ATPase and specificity subunit of ClpA-ClpP ATP-dependent serine protease, chaperone activity
Dickeya dadantii 3937 chromosome has lolA - chaperone

# Задание 3

In [30]:
data = ['GAACTCATGGTG', 
        'AAAAGCACGGTC',
        'TCAAAGCAAGGC',
        'CCTAATCAGGGC',
        'AAGTATGGACTC',
        'ACTAAGCAGGGT',
        'TCTCACGGCCCA',
        'CCTCGTGGTGGG',
        'TACCGTATGGTT',
        'ACCACTCGTCGA']
l = 8
start_pos = [2, 2, 3, 3, 0, 3, 0, 0, 2, 3]

In [31]:
for line, start in zip(data, start_pos):
    print('.' * (max(start_pos) - start) + line + '.' * (start - min(start_pos)))

.GAACTCATGGTG..
.AAAAGCACGGTC..
TCAAAGCAAGGC...
CCTAATCAGGGC...
...AAGTATGGACTC
ACTAAGCAGGGT...
...TCTCACGGCCCA
...CCTCGTGGTGGG
.TACCGTATGGTT..
ACCACTCGTCGA...
